# Query data

Learn three search methods: **Vector** (semantic similarity), **Keyword** (BM25), and **Hybrid** (combines both).

#### Get keys and urls

## Connect to Weaviate

Connect to a Weaviate instance.

In [ ]:
import weaviate
from weaviate.classes.init import Auth
# from weaviate.classes.init import AdditionalConfig, Timeout

# client = weaviate.connect_to_custom(
#     http_host="<http_host>",
#     http_port="<http_port>",
#     grpc_host="<grpc_host>",
#     grpc_port="<grpc_port>",
# )

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.environ["WCD_TEST_URL"],
    auth_credentials=os.environ["WCD_TEST_KEY"]
)

client.is_ready()

True

### Helper function

Get our collection of financial articles (5000 articles total).

In [45]:
articles = client.collections.get("FinancialArticles")

In [46]:
len(articles)

5000

## Vector search

![images/search_vector.png](images/search_vector.png)

[Docs - near_text](https://weaviate.io/developers/weaviate/search/similarity#an-input-medium)

In [47]:
response = articles.query.near_text(
    query="tech market trends",
    limit=5,
    target_vector="title"
)

for item in response.objects:
    print(item.properties["article_title"])

GLOBAL MARKETS-US tech stocks gain in mixed global moves as inflation lingers
2 Tech Stocks Riding Huge Megatrends
Buy 5 Tech Stocks at Attractive Valuation to Tap Market Rally
Market Split Continues With Chips, Leading Techs Retreating; S&P 500 Futures
Sector Update: Technology Stocks Pressured in Pre-Market


In [48]:
from weaviate.classes.query import MetadataQuery

response = articles.query.near_text(
    query="tech market trends",
    limit=5,
    target_vector="title",
    return_metadata=MetadataQuery(distance=True)
)

for item in response.objects:
    print(item.properties["article_title"])
    print(item.metadata.distance)

GLOBAL MARKETS-US tech stocks gain in mixed global moves as inflation lingers
0.5290522575378418
2 Tech Stocks Riding Huge Megatrends
0.5424867272377014
Buy 5 Tech Stocks at Attractive Valuation to Tap Market Rally
0.5463999509811401
Market Split Continues With Chips, Leading Techs Retreating; S&P 500 Futures
0.5475603938102722
Sector Update: Technology Stocks Pressured in Pre-Market
0.5492198467254639


## Filters

Add conditions to narrow down search results before querying.

### Fetch with filters

In [49]:
from weaviate.classes.query import Filter

response = articles.query.fetch_objects(
    limit=5,
    filters=Filter.by_property("article_title").like("apple")
)

print(f"Returned object count: {len(response.objects)}")

for item in response.objects:
    print(item.properties["article_title"])

Returned object count: 5
Qualcomm, Jabil: Should Apple's Suppliers Look Forward To A Strong 2021?
Apple Implications Overshadow Qualcomm’s Earnings Beat
Apple's Steady Results Prove Why It's the Ultimate Warren Buffett Stock
Thursday’s Vital Data: Intel, Apple and Nvidia
Buying This Fund Is Like Buying Apple With a 12.1% Dividend


In [50]:
from weaviate.classes.query import Filter

response = articles.query.fetch_objects(
    limit=5,
    filters=(
        Filter.by_property("article_title").like("aust*") &
        Filter.by_property("article_title").like("ev")
    )
)

print(f"Returned object count: {len(response.objects)}")

for item in response.objects:
    print(item.properties["article_title"])

Returned object count: 2
ANALYSIS-Australia's push for faster EV uptake will be slow to charge
BMW AG (BAMXF) Announces Ambitious EV Plans in China, Austria


## Search with filters

Combine vector search with filters for precise results.

In [51]:
from weaviate.classes.query import Filter

response = articles.query.near_text(
    query="strategy",
    target_vector="title",
    limit=5,
    filters=Filter.by_property("article_title").like("netflix")
)

print(f"Returned object count: {len(response.objects)}")

for item in response.objects:
    print(item.properties["article_title"])

Returned object count: 5
3 Key Ways Netflix Has Changed Its Content Strategy
Could This Be Netflix's Next Big Move?
PREVIEW-Netflix shielded from Hollywood strike by global crew, strong pipeline
Comcast Might Be the Key to Netflix's Continued Subscriber Growth
SA Stocks To Watch: Netflix To The Rescue?


## Keyword Search

![images/search_keyword.png](images/search_keyword.png)

[Docs - keyword/bm25](https://weaviate.io/developers/weaviate/search/bm25)

In [55]:
response = articles.query.bm25(
    query="earnings report",
    query_properties=["article_title"],
    limit=5,
)

for item in response.objects:
    print(item.properties["article_title"])

5 Big Takeaways From Costco's Earnings Report
My Take on AMD's Strong Earnings Report
Walgreens Boots Alliance to report earnings March 28
Here's What Stood Out in Starbucks' Earnings Report
1 Number To Watch in Alphabet's Earnings Report


In [56]:
from weaviate.classes.query import MetadataQuery

response = articles.query.bm25(
    query="earnings report",
    query_properties=["article_title"],
    limit=5,
    return_metadata=MetadataQuery(score=True)
)

for item in response.objects:
    print(item.properties["article_title"])
    print(item.metadata.score)

5 Big Takeaways From Costco's Earnings Report
2.764371156692505
My Take on AMD's Strong Earnings Report
2.764371156692505
Walgreens Boots Alliance to report earnings March 28
2.764371156692505
Here's What Stood Out in Starbucks' Earnings Report
2.6476149559020996
1 Number To Watch in Alphabet's Earnings Report
2.6476149559020996


In [57]:
from weaviate.classes.query import MetadataQuery

response = articles.query.bm25(
    query="earnings report",
    query_properties=["article_title", "article"],
    limit=5,
    return_metadata=MetadataQuery(score=True)
)

for item in response.objects:
    print(item.properties["article_title"])
    print(item.metadata.score)

Drug Stocks' Earnings Previews: KERX, BXLT, BMRN, AGEN, IMGN
2.3673558235168457
Taking Stock of the Q3 Earnings season - Analyst Blog
2.3612799644470215
Revenue Weakness Stands Out in Q3 Reports
2.3432424068450928
Drug Stocks Earnings Slated for Feb 14: INCY, PRTA & More
2.3295507431030273
Q4 Earnings Inform the Market - Ahead of Wall Street
2.3190536499023438


In [58]:
from weaviate.classes.query import MetadataQuery

response = articles.query.bm25(
    query="earnings report",
    query_properties=["article_title", "article^3"],
    limit=5,
    return_metadata=MetadataQuery(score=True)
)

for item in response.objects:
    print(item.properties["article_title"])
    print(item.metadata.score)

Analog Devices (ADI) Q4 Earnings to Ride on End Market Growth
6.443612098693848
Nokia (NOK) Solutions to Power Charter's Network Connectivity
6.225020885467529
Why Seagate (STX) Stock is a Must Have in Your Portfolio
6.199409484863281
Dollar Tree (DLTR) Stock Moves -0.48%: What You Should Know
6.085502624511719
Dow Takes Back 13,000, But Can't Reclaim Key Trendline Support
6.079561710357666


## Hybrid search

![images/search_hybrid.png](images/search_hybrid.png)

[Docs - hybrid](https://weaviate.io/developers/weaviate/search/hybrid)

In [60]:
response = articles.query.hybrid(
    query="earnings report",
    query_properties=["article_title"],
    limit=5,
    alpha=0.7,
    target_vector="title"
)

for item in response.objects:
    print(item.properties["article_title"])

Weekly Market Summary: Investors Digest Strong Earnings Reports
1 Number To Watch in Alphabet's Earnings Report
My Take on AMD's Strong Earnings Report
PREVIEW-Tesla investors to focus on demand issues in earnings report
5 Big Takeaways From Costco's Earnings Report


### Hybrid - Explain score

See how hybrid search combines vector and keyword scores.

In [ ]:
from weaviate.classes.query import MetadataQuery

response = articles.query.hybrid(
    query="earnings report",
    query_properties=["article_title"],
    limit=5,
    alpha=0.7,
    target_vector="title",
    return_metadata=MetadataQuery(score=True, explain_score=True)
)

for item in response.objects:
    print(item.properties["article_title"])
    print(item.metadata.score)
    print(item.metadata.explain_score)

Weekly Market Summary: Investors Digest Strong Earnings Reports
0.699999988079071

Hybrid (Result Set vector,hybridVector) Document dc5b84ad-fb6c-4b1f-a5cc-de45b53372b9: original score 0.45347238, normalized score: 0.7
1 Number To Watch in Alphabet's Earnings Report
0.6810106635093689

Hybrid (Result Set keyword,bm25) Document 14566206-b405-4336-8b61-a5b05f06b9d5: original score 2.647615, normalized score: 0.26749945 - 
Hybrid (Result Set vector,hybridVector) Document 14566206-b405-4336-8b61-a5b05f06b9d5: original score 0.39000446, normalized score: 0.41351122
My Take on AMD's Strong Earnings Report
0.6388599872589111

Hybrid (Result Set keyword,bm25) Document 50145e66-2b4e-42dd-bffd-0cbb17657775: original score 2.7643712, normalized score: 0.3 - 
Hybrid (Result Set vector,hybridVector) Document 50145e66-2b4e-42dd-bffd-0cbb17657775: original score 0.37346643, normalized score: 0.33885998
PREVIEW-Tesla investors to focus on demand issues in earnings report
0.6061513423919678

Hybrid (Re

## Close the client

Always close your connection when finished.

In [ ]:
client.close()